In [32]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from scipy.spatial.distance import cosine

# Load images
image1_path = "./dataset/train/wedo/wedo_9.jpeg"
image2_path = "./dataset/train/wedo/wedo_13.jpeg"

In [33]:
def histogram_similarity(image1, image2):
    # Convert images to grayscale
    hist_img1 = cv2.calcHist([image1], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
    hist_img1[255, 255, 255] = 0 #ignore all white pixels
    cv2.normalize(hist_img1, hist_img1, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    hist_img2 = cv2.calcHist([image2], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
    hist_img2[255, 255, 255] = 0  #ignore all white pixels
    cv2.normalize(hist_img2, hist_img2, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    # Calculate histogram intersection
    hist_intersection = cv2.compareHist(hist_img1, hist_img2, cv2.HISTCMP_CORREL)
    
    return hist_intersection

def ssim_similarity(image1, image2):
    # Convert images to grayscale
    image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]), interpolation = cv2.INTER_AREA)
    print(image1.shape, image2.shape)
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    
    # Calculate SSIM
    ssim_index, _ = ssim(gray1, gray2, full=True)
    
    return ssim_index

def mse_similarity(image1, image2):
    # Convert images to grayscale
    image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]), interpolation = cv2.INTER_AREA)
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    
    # Calculate MSE
    mse = np.mean((gray1 - gray2) ** 2)
    
    return mse

def psnr(image1, image2):
    image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]), interpolation = cv2.INTER_AREA)
    # Convert images to float32
    image1 = image1.astype(np.float32)
    image2 = image2.astype(np.float32)
    
    # Calculate squared difference
    squared_diff = (image1 - image2) ** 2
    
    # Calculate mean squared error
    mse = np.mean(squared_diff)
    
    # Calculate maximum pixel value
    max_pixel = np.max(image1)
    
    # Calculate PSNR
    psnr_value = 20 * np.log10(max_pixel / np.sqrt(mse))
    
    return psnr_value

def cosine_similarity(image1, image2):
    # Flatten images
    image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]), interpolation = cv2.INTER_AREA)
    flat_image1 = image1.flatten()
    flat_image2 = image2.flatten()
    
    # Calculate cosine similarity
    cosine_sim = 1 - cosine(flat_image1, flat_image2)
    
    return cosine_sim

In [34]:
# Load images
image1 = cv2.imread(image1_path)
image2 = cv2.imread(image2_path)

# Calculate similarities
hist_similarity = histogram_similarity(image1, image2)
ssim_similarity_val = ssim_similarity(image1, image2)
mse_similarity_val = mse_similarity(image1, image2)
cosine_similarity_val = cosine_similarity(image1, image2)
psnr_value = psnr(image1, image2)

print("Histogram Similarity:", hist_similarity)
print("SSIM Similarity:", ssim_similarity_val)
print("MSE Similarity:", mse_similarity_val)
print("Cosine Similarity:", cosine_similarity_val)
print("PSNR:", psnr_value)

(1800, 2034, 3) (1800, 2034, 3)
Histogram Similarity: 0.0541776724357846
SSIM Similarity: 0.37704129916562235
MSE Similarity: 104.66067027204195
Cosine Similarity: 0.8932247369323221
PSNR: 11.009584665298462


In [35]:
import torch
import open_clip
import cv2
from sentence_transformers import util
from PIL import Image
# image processing model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16-plus-240', pretrained="laion400m_e32")
model.to(device)
def imageEncoder(img):
    img1 = Image.fromarray(img).convert('RGB')
    img1 = preprocess(img1).unsqueeze(0).to(device)
    img1 = model.encode_image(img1)
    return img1
def generateScore(image1, image2):
    test_img = cv2.imread(image1_path, cv2.IMREAD_UNCHANGED)
    data_img = cv2.imread(image2_path, cv2.IMREAD_UNCHANGED)
    img1 = imageEncoder(test_img)
    img2 = imageEncoder(data_img)
    cos_scores = util.pytorch_cos_sim(img1, img2)
    score = round(float(cos_scores[0][0])*100, 2)
    return score
print(f"similarity Score: ", round(generateScore(image1, image2), 2))

similarity Score:  66.26
